In [26]:
import itertools
import pathlib as pl

import nltk.data
from nltk.tokenize import RegexpTokenizer
import pandas as pd
from tqdm import tqdm

from pyprojroot import here

paper_df = pd.read_csv(here() / "data" / "db" / "final" / "kaggle" / "paper_text" / "document_parses_pmc_json.tsv", sep = "\t")

# can also load within the here function
# paper_df = pd.read_csv(here("./data/db/final/kaggle/paper_text/document_parses_pmc_json.tsv"), sep = "\t")

search_terms = [
    "incubation period",
    "infectiousness period",
    "recovery rate",
    "case fatality ratio",
    "asymptomatic fraction",
    "hospitalized fraction"
]

In [22]:
def has_search_terms(dat, term, text_col):
    term_sentences = dat[text_col].str.contains(term, case=False, regex=True)
    new_term_string = term.replace(" ", "_")
    dat[f"has-{new_term_string}"] = term_sentences
    return(dat)

In [27]:
has_terms_df = paper_df

for term in tqdm(search_terms):
    has_terms_df = has_search_terms(has_terms_df, term, 'text')

100%|█████████████████████████████████████████████████████████████████████| 6/6 [01:09<00:00, 11.60s/it]


,pid,num_authors,title,text,has-incubation_period,has-infectiousness_period,has-recovery_rate,has-case_fatality_ratio,has-asymptomatic_fraction,has-hospitalized_fraction
0,PMC1054884,7,Recombination Every Day: Abundant Recombinatio...,As increasing numbers of full-length viral seq...,False,False,False,False,False,False
1,PMC1065028,1,Why can't I visit? The ethics of visitation re...,The sudden emergence of severe acute respirato...,False,False,False,False,False,False
2,PMC1065064,8,Prospective evaluation of an internet-linked h...,The rate of expansion of medical knowledge is ...,False,False,False,False,False,False
3,PMC1065120,4,Scanning the horizon: emerging hospital-wide t...,This series of articles provides regular surve...,False,False,False,False,False,False
4,PMC1065257,3,Characterization of the frameshift signal of E...,Programmed −1 ribosomal frameshifting (hereaft...,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...
33498,PMC7204586,2,A Call for Randomized Controlled Trials to Tes...,Novel coronavirus disease (COVID-19) is spread...,False,False,False,False,False,False
33499,PMC7204595,28,Advanced Preparation Makes Research in Emergen...,Nebraska’s first guests related to COVID-19 we...,False,False,False,False,False,False
33500,PMC7204610,1,Olympics in the Time of a Pandemic,The Olympics of the modern era have establishe...,False,False,False,False,False,False
33501,PMC7204696,2,A glimpse into the eye of the COVID-19 cytokin...,The authors declare no conflict of interest.,False,False,False,False,False,False


In [42]:
display(has_terms_df
     .filter(like="has-", axis='columns')
     .apply(lambda x: x.value_counts(dropna=False))
     .assign(count=lambda x: x.apply('sum', axis='columns'))
)

,has-incubation_period,has-infectiousness_period,has-recovery_rate,has-case_fatality_ratio,has-asymptomatic_fraction,has-hospitalized_fraction,count
False,30983,33496,33139,33376,33501,33502,197997
True,2520,7,364,127,2,1,3021


In [ ]:
# break up into sentences
#import nltk
#nltk.download('punkt')
sent_detector = nltk.data.load('tokenizers/punkt/english.pickle')

has_terms_df["text_sent"] = has_terms_df["text"].apply(lambda x: sent_detector.tokenize(x.strip()))


# save out working data
pl.Path(here("./data/db/working/kaggle/id_model_inputs", warn=False)).mkdir(parents=True, exist_ok=True)
has_terms_df.to_pickle(here("./data/db/working/kaggle/id_model_inputs/01-sentences-keywords.pickle", warn=False))
